Lectura de xml

In [1]:
import pandas as pd
import xml.etree.ElementTree as etree



#tree = etree.parse("tg-fal17.xml")
tree = etree.parse("agh-ggis-spr17.xml")
root = tree.getroot()

columns_problem = ["Name", "NrDays", "SlotsPerDay", "NrWeeks"]
columns_optimization = ["Time", "Room", "Distribution", "Student"]
columns_rooms = ["RoomId", "Capacity", "PreviousRoomIdList", "TravelTimeValueList", "UnavailableDaysList", "UnavailableStartList", "UnavailableLengthList", "UnavailableWeeksList"]
columns_courses = ["CourseId", "ConfigIdList"]
columns_configs = ["ConfigId", "SubpartIdList"]
columns_subparts = ["SubpartId", "ClassIdList"]
columns_classes = ["ClassId", "ClassLimit", "ClassParent"]
columns_distributions = ["DistributionId", "DistributionType", "DistributionRequired", "DistributionPenalty", "ClassesList"]
columns_students = ["StudentId", "CoursesList"]
columns_relation_class_room = ["ClassId", "RoomId", "RoomPenalty"]
columns_relation_class_time = ["ClassId", "TimeDays", "TimeStart", "TimeLength", "TimeWeeks", "TimePenalty"]

df_problem = pd.DataFrame(columns = columns_problem)
df_optimization = pd.DataFrame(columns = columns_optimization)
df_rooms = pd.DataFrame(columns = columns_rooms)
df_courses = pd.DataFrame(columns = columns_courses)
df_configs = pd.DataFrame(columns = columns_configs)
df_subparts = pd.DataFrame(columns = columns_subparts)
df_classes = pd.DataFrame(columns = columns_classes)
df_distributions = pd.DataFrame(columns = columns_distributions)
df_students = pd.DataFrame(columns = columns_students)
df_relation_class_room = pd.DataFrame(columns = columns_relation_class_room)
df_relation_class_time = pd.DataFrame(columns = columns_relation_class_time)




df_problem = df_problem.append({"Name": root.attrib.get("name"), "NrDays": root.attrib.get("nrDays"), "SlotsPerDay": root.attrib.get("slotsPerDay"), "NrWeeks": root.attrib.get("nrWeeks")}, ignore_index = True)

for node in root:
    if node.tag == "optimization":
        df_optimization = df_optimization.append({"Time": node.attrib.get("time"), "Room": node.attrib.get("room"), "Distribution": node.attrib.get("distribution"), "Student": node.attrib.get("student")}, ignore_index = True)
    
    elif node.tag == "rooms":
        for room in node:
            roomid = room.attrib.get("id")
            cap = room.attrib.get("capacity")
            prevroomlist = []
            valuelist = []
            dayslist = []
            startlist = []
            lengthlist = []
            weekslist = []
            if len(list(child.tag for child in room.iter() if child is not room)) > 0:
                for child in room:
                    if child.tag == "travel":
                        prevroomid = child.attrib.get("room")
                        value = child.attrib.get("value")
                        prevroomlist.append(prevroomid)
                        valuelist.append(value)
                    elif child.tag == "unavailable":
                        days = child.attrib.get("days")
                        start = child.attrib.get("start")
                        length = child.attrib.get("length")
                        weeks = child.attrib.get("weeks")
                        dayslist.append(days)
                        startlist.append(start)
                        lengthlist.append(length)
                        weekslist.append(weeks)
            if len(prevroomlist) == 0:
                prevroomlist = None
            if len(valuelist) == 0:
                valuelist = None
            if len(dayslist) == 0:
                dayslist = None
            if len(startlist) == 0:
                startlist = None
            if len(lengthlist) == 0:
                lengthlist = None
            if len(weekslist) == 0:
                weekslist = None
            df_rooms = df_rooms.append(pd.Series([roomid, cap, prevroomlist, valuelist, dayslist, startlist, lengthlist, weekslist], index = columns_rooms), ignore_index = True)
            
    elif node.tag == "courses":
        for course in node:
            #print(course.tag)
            courseid = course.attrib.get("id")
            configlist = []
            for config in course:
                #print(config.tag)
                configid = config.attrib.get("id")
                configlist.append(configid)
                subpartlist = []
                for subpart in config:
                    #print(subpart.tag)
                    subpartid = subpart.attrib.get("id")
                    subpartlist.append(subpartid)
                    claselist = []
                    for clase in subpart:
                        #print(clase.tag)
                        claseid = clase.attrib.get("id")
                        claselist.append(claseid)
                        claselimit = clase.attrib.get("limit")
                        claseparent = clase.attrib.get("parent") if clase.attrib.get("parent") is not None else None
                        df_classes = df_classes.append(pd.Series([claseid, claselimit, claseparent], index = columns_classes), ignore_index = True)
                        if len(list(child.tag for child in clase.iter() if child is not clase)) > 0:
                            for child in clase:
                                #print(child.tag)
                                if child.tag == "room":
                                    roomid = child.attrib.get("id")
                                    roompenalty = child.attrib.get("penalty")
                                    df_relation_class_room = df_relation_class_room.append(pd.Series([claseid, roomid, roompenalty], index = columns_relation_class_room), ignore_index = True)
                                elif child.tag == "time":
                                    timedays = child.attrib.get("days")
                                    timestart = child.attrib.get("start")
                                    timelength = child.attrib.get("length")
                                    timeweeks = child.attrib.get("weeks")
                                    timepenalty = child.attrib.get("penalty")
                                    df_relation_class_time = df_relation_class_time.append(pd.Series([claseid, timedays, timestart, timelength, timeweeks, timepenalty], index = columns_relation_class_time), ignore_index = True)
                    df_subparts = df_subparts.append(pd.Series([subpartid, claselist], index = columns_subparts), ignore_index = True)
                df_configs = df_configs.append(pd.Series([configid, subpartlist], index = columns_configs), ignore_index = True)
            df_courses = df_courses.append(pd.Series([courseid, configlist], index = columns_courses), ignore_index = True)
                                    
    elif node.tag == "distributions":
        contador = 1
        for distribution in node:
            #print(distribution.tag)
            distribid = contador
            distribtype = distribution.attrib.get("type")
            distribrequired = distribution.attrib.get("required") if distribution.attrib.get("required") is not None else None
            distribpenalty = distribution.attrib.get("penalty") if distribution.attrib.get("penalty") is not None else None
            listaclases = []
            for clase in distribution:
                #print(clase.tag)
                listaclases.append(clase.attrib.get("id"))
            df_distributions = df_distributions.append(pd.Series([distribid, distribtype, distribrequired, distribpenalty, listaclases], index = columns_distributions), ignore_index = True)
            contador += 1
             
    elif node.tag == "students":
        for student in node:
            studentid = student.attrib.get("id")
            listacourses = []
            for course in student:
                listacourses.append(course.attrib.get("id"))
            df_students = df_students.append(pd.Series([studentid, listacourses], index = columns_students), ignore_index = True)
            
            


print("################################################################################################################")
print("DataFrame Problem")
display(df_problem)
print("################################################################################################################")
print("DataFrame Optimization")
display(df_optimization)
print("################################################################################################################")
print("DataFrame Rooms")
display(df_rooms)
print("################################################################################################################")
print("DataFrame Courses")
display(df_courses)
print("################################################################################################################")
print("DataFrame Configs")
display(df_configs)
print("################################################################################################################")
print("DataFrame Subparts")
display(df_subparts)
print("################################################################################################################")
print("DataFrame Classes")
display(df_classes)
print("################################################################################################################")
print("DataFrame Distributions")
display(df_distributions)
print("################################################################################################################")
print("DataFrame Students")
display(df_students)
print("################################################################################################################")
print("DataFrame RelationClassRoom")
display(df_relation_class_room)
print("################################################################################################################")
print("DataFrame RelationClassTime")
display(df_relation_class_time)
print("################################################################################################################")

################################################################################################################
DataFrame Problem


,Name,NrDays,SlotsPerDay,NrWeeks
0,agh-ggis-spr17,7,288,16


################################################################################################################
DataFrame Optimization


,Time,Room,Distribution,Student
0,4,1,15,5


################################################################################################################
DataFrame Rooms


...

################################################################################################################
DataFrame RelationClassRoom


,ClassId,RoomId,RoomPenalty
0,1,43,0
1,2,43,0
2,3,43,0
3,4,43,0
4,5,43,0
...,...,...,...
12372,1833,32,0
12373,1835,24,0
12374,1838,24,0
12375,1839,24,0


################################################################################################################
DataFrame RelationClassTime


,ClassId,TimeDays,TimeStart,TimeLength,TimeWeeks,TimePenalty
0,1,1000000,96,18,0101010010010101,0
1,1,0100000,96,18,0101010010010101,0
2,1,0010000,96,18,0101010010010101,0
3,1,0000100,96,18,0101010010101010,0
4,1,1000000,106,18,0101010010010101,0
...,...,...,...,...,...,...
46662,1836,0100000,176,18,0101010010010101,0
46663,1837,0100000,156,18,1010101000101010,0
46664,1838,0100000,176,18,0101010010010101,0
46665,1839,0100000,156,18,1010101000101010,0


################################################################################################################


Lectura xml solucion

In [2]:
import pandas as pd
import xml.etree.ElementTree as etree

tree = etree.parse("solution-tg-fal17.xml")
root = tree.getroot()

columns_sol = ["ClassId", "Days", "Start", "Weeks", "Room"]

df_sol = pd.DataFrame(columns = columns_sol)

for clase in root:
    

SyntaxError: unexpected EOF while parsing (<ipython-input-2-2edd5ad0955c>, line 12)

Analisis de datos

In [6]:
## Duracion en slots de clases:

df_relation_class_time["TimeLength"] = df_relation_class_time["TimeLength"].astype(int)
print("Promedio:")
display(df_relation_class_time["TimeLength"].mean())
print("Min:")
display(df_relation_class_time["TimeLength"].min())
print("Max:")
display(df_relation_class_time["TimeLength"].max())
print("Tiempo de inicio")
display(df_relation_class_time.groupby("TimeStart").count())
print("Count")
display(df_relation_class_time["TimeLength"].value_counts())
print("Mediana:")
display(df_relation_class_time["TimeLength"].median())
print("Cuartiles:")
display(df_relation_class_time["TimeLength"].quantile([0, 0.25, 0.5, 0.75, 1]))
print("Var:")
display(df_relation_class_time["TimeLength"].var())
print("Std:")
display(df_relation_class_time["TimeLength"].std())

Promedio:


22.044592538624723

Min:


9

Max:


...

0.00      9.0
0.25     18.0
0.50     18.0
0.75     27.0
1.00    144.0
Name: TimeLength, dtype: float64

Var:


30.098277180969482

Std:


5.486189677815513

In [2]:
## Disponibilidad de salas:

for index, row in df_rooms.iterrows():
    print(row["RoomId"], row["UnavailableDaysList"])
    #if row["UnavailableDaysList"] is None:
    #    print(row["RoomId"], row["UnavailableDaysList"])

1 ['1000000', '0100000', '0100000', '0010000', '0001000', '0001000', '0000100', '0000010', '0000001']
2 ['1000000', '0100000', '0010000', '0001000', '0000100', '0000010', '0000010', '0000001']
3 ['1000000', '0100000', '0010000', '0001000', '0000100', '0000010', '0000010', '0000001', '0000001']
4 ['0000100', '0000010', '0000010', '0000001', '0000001', '1000000', '1000000', '0100000', '0001000', '0000100', '0000100', '0000100', '0000100', '0000100', '0000010', '0000010', '0000010', '0000010', '0000001', '0000001', '0000001', '0000001', '0000001']
5 ['0000100', '0000010', '0000010', '0000001', '0000001', '0100000', '0010000', '0000100', '0000100', '0000100', '0000100', '0000010', '0000010', '0000010', '0000010', '0000001']
6 ['1000000', '0100000', '0010000', '0001000', '0000100', '0000010', '0000001']
7 ['1000000', '0100000', '0010000', '0010000', '0001000', '0000100', '0000010', '0000001']
8 ['1000000', '1000000', '1000000', '0100000', '0010000', '0001000', '0000100', '0000010', '0000001

In [8]:
## Variabilidad de clases:

#for index, row in df_relation_class_time.iterrows():
#    print(row["ClassId"], row["TimeDays"])
display(df_relation_class_time["ClassId"].value_counts())

400     520
628     160
593     160
629     160
311     128
       ... 
168       1
726       1
1815      1
365       1
1777      1
Name: ClassId, Length: 1852, dtype: int64

In [ ]:
#################### Nueva Parte versión Pau #######################
print("Slots de tiempo | Cantidad de cursos que duran esos slots")
dic={}
list=[]
for index, row in df_relation_class_time.iterrows():
    #print(row["ClassId"], row["TimeLength"])
    if row["TimeLength"] not in dic.keys():
        dic[row["TimeLength"]] = [0,[]]
    if row["ClassId"] not in dic[row["TimeLength"]][1]:
        #print(row["TimeLength"],row["ClassId"], dic[row["TimeLength"]][1])
        dic[row["TimeLength"]][0]+=1
        dic[row["TimeLength"]][1].append(row["ClassId"])
c= 0
for i in dic:
    print(i, dic[i][0])
    c += dic[i][0]
print("Total",c)

In [11]:
#Agrupar capacidad de salas


print("Capacidad | Cantidad de Salas")
display(df_rooms.groupby("Capacity").count()["RoomId"])

Capacidad | Cantidad de Salas


Capacity
120     1
140     1
15      9
180     2
240     1
25      1
28      2
29      1
30     13
35      3
42      3
45      1
70      1
90      3
94      1
96      1
Name: RoomId, dtype: int64

In [98]:
#Agrupar capacidad de clases

print("Límite de las clases | Cantidad de clases con el límite")
display(df_classes.groupby("ClassLimit").count()["ClassId"])


Límite de las clases | Cantidad de clases con el límite


ClassLimit
10     495
100     12
20      70
24      45
35      28
56      33
95      28
Name: ClassId, dtype: int64

In [29]:
#Clases en horario similar -3 slots- de margen
#Tiempo inicio | Frecuencia de clases en ese horario | Id de clase
dicct = {}
df_relation_class_time["TimeStart"] = df_relation_class_time["TimeStart"].astype(int)
for index, row in df_relation_class_time.sort_values('TimeStart').iterrows():
    #print(row["ClassId"], row["TimeStart"])
    x1 = row["TimeStart"]-1
    x2 = row["TimeStart"]-2
    x3 = row["TimeStart"]-3
    if x1 in dicct.keys():
        if row["ClassId"] not in dicct[x1][1]:
            dicct[x1][1].append(row["ClassId"])
            dicct[x1][0] += 1
    elif x2 in dicct.keys():
        if row["ClassId"] not in dicct[x2][1]:
            dicct[x2][1].append(row["ClassId"])
            dicct[x2][0] += 1
    elif x3 in dicct.keys():
        if row["ClassId"] not in dicct[x3][1]:
            dicct[x3][1].append(row["ClassId"])
            dicct[x3][0] += 1
    elif row["TimeStart"] not in dicct.keys():
        dicct[row["TimeStart"]] = [0,[]]
        dicct[row["TimeStart"]][1].append(row["ClassId"])
        dicct[row["TimeStart"]][0] += 1
    elif row["TimeStart"] in dicct.keys() and row["ClassId"] not in dicct[row["TimeStart"]][1]:
        dicct[row["TimeStart"]][1].append(row["ClassId"])
        dicct[row["TimeStart"]][0] += 1

print("Tiempo inicio | Frecuencia de clases en ese horario | Lista de clase")
for i in dicct:
    print(i, dicct[i][0])

Tiempo inicio | Frecuencia de clases en ese horario | Lista de clase
93 1446
99 1
105 1349
111 2
116 1483
120 1
126 1440
132 7
136 1471
141 6
146 1437
150 9
156 1472
162 1
166 1362
171 5
176 1369
180 1
186 1327
190 2
195 1326
201 23
206 1242
210 2
216 1239
222 1
226 1203
231 1
236 1213
240 1
246 610
256 3


In [149]:
#Cantidad de clases en una subparte
c = 0
dics = {}
dicc_s = {}
for index, row in df_subparts.iterrows():
    dicc_s[row["SubpartId"]] = row["ClassIdList"]
    
for index, col in df_configs.iterrows():
    for i in col["SubpartIdList"]:
        if len(dicc_s[i]) not in dics.keys():
            dics[len(dicc_s[i])] = [1, [], [int(col["ConfigId"])]]
            for z in dicc_s[i]:
                dics[len(dicc_s[i])][1].append(z)
                dics[len(dicc_s[i])][0]+=1
        else:
            dics[len(dicc_s[i])][2].append(int(col["ConfigId"]))
            for z in dicc_s[i]:
                dics[len(dicc_s[i])][1].append(z)
                dics[len(dicc_s[i])][0]+=1


print("Cantidad de clases en una subparte | Cantidad de veces que se repite en las subpartes | Lista de clases | Config a la que pertenece ")
for i in sorted(dics):
    print(i, dics[i][0])

Cantidad de clases en una subparte | Cantidad de veces que se repite en las subpartes | Lista de clases | Config a la que pertenece 
1 882
2 451
3 121
4 201
5 56
6 37
8 25
10 51
12 37


In [157]:
cluster = {}
for i in sorted(dics):
    #print(dics[i][1])
    for j in dics[i][1]:
        #print(i, j)
        for x in dicct:
            #print(dicct[x][1][0])
            if j in dicct[x][1]:
                #print("HOLA")
                tupla2 = tuple(set(dics[i][2]))
                key = (tupla2,i,x)
                if key not in cluster.keys():
                    cluster[key] = []
                cluster[key].append(j)
                
print("(Cursos, cantidad de clases de subparte, horario): Clases que se adaptan")
print(cluster)



(Cursos, cantidad de clases de subparte, horario): Clases que se adaptan
{((1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 42, 43, 44, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 58, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 21

In [65]:
#Patrones de salas
diccr = {}
dic2 = {}
for index, row in df_relation_class_room.iterrows():
    #print(row["ClassId"], row["RoomId"])
    if row["ClassId"] not in diccr.keys():
        diccr[row["ClassId"]] = []
    diccr[row["ClassId"]].append(row["RoomId"])
#print(diccr)

for i in diccr:
    #print(i, diccr[i])
    tupla = tuple(diccr[i])
    if tupla not in dic2.keys():
        dic2[tupla] = []
    dic2[tupla].append(i)
#print(dic2)

c = 0

print("Salas | Preferencias de clases por la sala")
for i in dic2:
    print(i, dic2[i])
    c+=1
#print(c)

Salas | Preferencias de clases por la sala
('43',) ['1', '2', '3', '4', '5', '6', '7', '8']
('29',) ['9', '21', '34', '61', '90', '101', '117', '124', '125', '144', '145', '156', '182', '208', '379', '380', '381', '382', '383', '384', '385', '726', '1803', '1804', '1805', '1806', '1807', '1808', '1813', '1814', '1815', '1816', '1817', '1818']
('19', '38', '18', '36', '40', '41', '28', '32') ['10', '11', '12', '13', '14', '15', '189', '217', '313', '324']
('10', '23', '24', '29', '42') ['16', '77', '163', '402', '408', '414', '457', '465', '503', '508']
('34',) ['17', '18', '19', '20', '82', '83', '84', '85', '186', '187', '190', '192', '196', '197', '198', '199', '202', '579', '580', '581', '582', '628', '629', '1071', '1072', '1073', '1074', '1075', '1076', '1077', '1078', '1079', '1080', '1081', '1082', '1083', '1084', '1085', '1086', '1087', '1088', '1089', '1090', '1091', '1092', '1093', '1094', '1095', '1099', '1100', '1101', '1102', '1103', '1104', '1105', '1106']
('25', '18', '3

In [151]:
import math
#Agrupar capacidad de cursos

#print("Límite de las clases | Cantidad de clases con el límite")
#display(df_classes.groupby("ClassLimit").count()["ClassId"])

dicc_cursos = {}
#display(df_courses)
for index, row in df_courses.iterrows():
    dicc_cursos[row["CourseId"]] = row["ConfigIdList"]
#print(dicc_cursos)      
dicc_configs = {}
#display(df_configs)
for index, row in df_configs.iterrows():
    dicc_configs[row["ConfigId"]] = row["SubpartIdList"]
#print(dicc_configs)
dicc_subparts = {}
#display(df_subparts)
for index, row in df_subparts.iterrows():
    dicc_subparts[row["SubpartId"]] = row["ClassIdList"]
#print(dicc_subparts)
dicc_classes = {}
#display(df_classes)
for index, row in df_classes.iterrows():
    dicc_classes[row["ClassId"]] = row["ClassLimit"]
#print(dicc_classes)

dicc_lim = {}
for p,i in dicc_cursos.items():
    for j in i:
        if j in dicc_configs.keys():
            #print(dicc_configs[j])
            for k in dicc_configs[j]:
                if k in dicc_subparts.keys():
                    #print(dicc_subparts[k])
                    for l in dicc_subparts[k]:
                        #print(l)
                        if l in dicc_classes.keys():
                            n = int(dicc_classes[l])
                            if p not in dicc_lim.keys():
                                dicc_lim[p] = [n]
                            else:
                                dicc_lim[p].append(n)
#print(dicc_lim)
print("Id de curso | Limite de alumnos")
for i in dicc_lim.keys():
    suma = 0
    contador = 0
    for j in dicc_lim[i]:
        suma += j
        contador += 1
    promedio = suma/contador
    dicc_lim[i] = int(math.ceil(promedio/10.0))*10
    print(i, dicc_lim[i])

Id de curso | Limite de alumnos
1 20
2 20
3 20
4 60
5 50
6 30
7 30
8 30
9 30
10 60
11 50
12 130
13 120
14 30
15 30
16 60
17 130
18 30
19 180
20 50
21 50
22 60
23 60
24 50
25 20
26 20
27 20
28 30
29 30
30 30
31 30
32 30
33 30
34 30
35 30
36 30
37 30
38 40
39 60
40 50
41 30
42 60
43 60
44 40
45 30
46 60
47 60
48 40
49 60
50 20
51 30
52 20
53 40
54 30
55 30
56 30
57 30
58 30
59 30
60 40
61 30
62 30
63 30
64 20
65 30
66 30
67 40
68 30
69 20
70 30
71 30
72 40
73 40
74 30
75 30
76 30
77 20
78 30
79 20
80 30
81 30
82 30
83 30
84 20
85 40
86 30
87 40
88 40
89 30
90 30
91 60
92 40
93 30
94 50
95 10
96 60
97 180
98 30
99 50
100 30
101 30
102 40
103 40
104 80
105 30
106 30
107 50
108 30
109 40
110 120
111 120
112 40
113 40
114 40
115 80
116 30
117 40
118 60
119 40
120 80
121 40
122 30
123 40
124 40
125 50
126 20
127 50
128 40
129 60
130 50
131 40
132 30
133 40
134 30
135 40
136 50
137 40
138 30
139 50
140 50
141 50
142 40
143 30
144 50
145 20
146 30
147 30
148 30
149 50
150 30
151 50
152 30
153 3

In [153]:
## Cuantos cursos hay para cada capacidad redondeada

print("Capacidad de curso | Conteo de cursos")
dicc_cont = {}
for i in dicc_lim.values():
        if i not in dicc_cont.keys():
            dicc_cont[i] = 1
        else:
            dicc_cont[i] += 1
for i in dicc_cont.keys():
    print(i, dicc_cont[i])

Capacidad de curso | Conteo de cursos
20 21
60 20
50 35
30 128
130 6
120 3
180 2
40 46
10 1
80 8
70 2


In [146]:
## Cuantos cursos hay para cada capacidad redondeada

print("Capacidad de curso | Conteo de cursos | Cursos")
dicc_cont2 = {}
for i in dicc_lim:
    if dicc_lim[i] not in dicc_cont2.keys():
        dicc_cont2[dicc_lim[i]] = [int(i)]
    else:
        #dicc_cont2[dicc_lim[i]][0] += 1
        dicc_cont2[dicc_lim[i]].append(int(i))
for i in dicc_cont2.keys():
    print(i, dicc_cont2[i])

Capacidad de curso | Conteo de cursos | Cursos
20 [1, 2, 3, 25, 26, 27, 50, 52, 64, 69, 77, 79, 84, 126, 145, 180, 196, 207, 211, 217, 228]
60 [4, 10, 16, 22, 23, 39, 42, 43, 46, 47, 49, 91, 96, 118, 129, 183, 187, 201, 270, 271]
50 [5, 11, 20, 21, 24, 40, 94, 99, 107, 125, 127, 130, 136, 139, 140, 141, 144, 149, 151, 155, 158, 159, 160, 186, 190, 195, 215, 220, 232, 235, 241, 258, 259, 260, 261]
30 [6, 7, 8, 9, 14, 15, 18, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 41, 45, 51, 54, 55, 56, 57, 58, 59, 61, 62, 63, 65, 66, 68, 70, 71, 74, 75, 76, 78, 80, 81, 82, 83, 86, 89, 90, 93, 98, 100, 101, 105, 106, 108, 116, 122, 132, 134, 138, 143, 146, 147, 148, 150, 152, 153, 154, 156, 157, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 174, 178, 179, 182, 191, 194, 202, 203, 204, 205, 206, 208, 209, 210, 212, 213, 214, 216, 218, 219, 221, 222, 223, 224, 226, 230, 234, 237, 238, 239, 240, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 264, 266, 267, 268, 269, 272]
130 [12

In [158]:
cluster2 = {}
c = 0
for i in cluster:
    cluster2[(i[1],i[2])] = list(i[0])
print(cluster2)

{(1, 93): [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 42, 43, 44, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 58, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 215, 216, 217, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 22

In [155]:
print("Subparte | horario | límite clase")
cluster_cursos = {}
for i in cluster2:
    for j in dicc_cont2:
        c = 0
        for k in dicc_cont2[j]:
            if k in cluster2[i]:
                c+=1
        if c == len(dicc_cont2[j]):
            cluster_cursos[(i[0],i[1],j)] = dicc_cont2[j]

print(cluster_cursos)


Subparte | horario | límite clase
{(1, 93, 20): [1, 2, 3, 25, 26, 27, 50, 52, 64, 69, 77, 79, 84, 126, 145, 180, 196, 207, 211, 217, 228], (1, 93, 60): [4, 10, 16, 22, 23, 39, 42, 43, 46, 47, 49, 91, 96, 118, 129, 183, 187, 201, 270, 271], (1, 93, 130): [12, 17, 175, 177, 192, 199], (1, 93, 120): [13, 110, 111], (1, 93, 180): [19, 97], (1, 93, 10): [95], (1, 93, 80): [104, 115, 120, 171, 172, 184, 197, 200], (1, 93, 70): [262, 263], (1, 105, 20): [1, 2, 3, 25, 26, 27, 50, 52, 64, 69, 77, 79, 84, 126, 145, 180, 196, 207, 211, 217, 228], (1, 105, 60): [4, 10, 16, 22, 23, 39, 42, 43, 46, 47, 49, 91, 96, 118, 129, 183, 187, 201, 270, 271], (1, 105, 130): [12, 17, 175, 177, 192, 199], (1, 105, 120): [13, 110, 111], (1, 105, 180): [19, 97], (1, 105, 10): [95], (1, 105, 80): [104, 115, 120, 171, 172, 184, 197, 200], (1, 105, 70): [262, 263], (1, 116, 20): [1, 2, 3, 25, 26, 27, 50, 52, 64, 69, 77, 79, 84, 126, 145, 180, 196, 207, 211, 217, 228], (1, 116, 60): [4, 10, 16, 22, 23, 39, 42, 43, 46

In [156]:
lista = []
c=0
for i in cluster_cursos:
    #print(cluster_cursos[i])
    for j in cluster_cursos[i]:
        if j not in lista:
            lista.append(j)
            c+=1
print(cluster_cursos)
print(c)
print(sorted(lista))

{(1, 93, 20): [1, 2, 3, 25, 26, 27, 50, 52, 64, 69, 77, 79, 84, 126, 145, 180, 196, 207, 211, 217, 228], (1, 93, 60): [4, 10, 16, 22, 23, 39, 42, 43, 46, 47, 49, 91, 96, 118, 129, 183, 187, 201, 270, 271], (1, 93, 130): [12, 17, 175, 177, 192, 199], (1, 93, 120): [13, 110, 111], (1, 93, 180): [19, 97], (1, 93, 10): [95], (1, 93, 80): [104, 115, 120, 171, 172, 184, 197, 200], (1, 93, 70): [262, 263], (1, 105, 20): [1, 2, 3, 25, 26, 27, 50, 52, 64, 69, 77, 79, 84, 126, 145, 180, 196, 207, 211, 217, 228], (1, 105, 60): [4, 10, 16, 22, 23, 39, 42, 43, 46, 47, 49, 91, 96, 118, 129, 183, 187, 201, 270, 271], (1, 105, 130): [12, 17, 175, 177, 192, 199], (1, 105, 120): [13, 110, 111], (1, 105, 180): [19, 97], (1, 105, 10): [95], (1, 105, 80): [104, 115, 120, 171, 172, 184, 197, 200], (1, 105, 70): [262, 263], (1, 116, 20): [1, 2, 3, 25, 26, 27, 50, 52, 64, 69, 77, 79, 84, 126, 145, 180, 196, 207, 211, 217, 228], (1, 116, 60): [4, 10, 16, 22, 23, 39, 42, 43, 46, 47, 49, 91, 96, 118, 129, 183, 1